In [1]:
import pandas
from bs4 import BeautifulSoup
import splinter
import re

In [2]:
monsters_to_import = [
    'http://dndroll.wikidot.com/creatures:giant-poisonous-snake',
    'http://dndroll.wikidot.com/creatures:ape',
    'http://dndroll.wikidot.com/creatures:spider'
]

In [3]:
url = monsters_to_import[2]
monster = {}
monster['url'] = url
browser = splinter.Browser("firefox")
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
browser.quit()

Problem reading geckodriver versions: error sending request for url (https://raw.githubusercontent.com/SeleniumHQ/selenium/trunk/common/geckodriver/geckodriver-support.json): error trying to connect: dns error: failed to lookup address information: Try again. Using latest geckodriver version
There was an error managing geckodriver (error sending request for url (https://github.com/mozilla/geckodriver/releases/latest): error trying to connect: dns error: failed to lookup address information: Try again); using driver found in the cache
Error sending stats to Plausible: error sending request for url (https://plausible.io/api/event): operation timed out


In [20]:
main_block = soup.find("div", id="main-content")
name = main_block.find("div", id="page-title").text.strip()
stats_table = main_block.find("table")
meta = None
AC = None
HP = None
movement = None

def get_AC(line):
    return re.match(r'[^0-9]*([0-9]+)', line).group(1)
def get_HP(line):
    return re.match(r'[^0-9]*(.*)', line).group(1)

page_block = main_block.find("div", id="page-content")
if page_block.find("p").findNext("table") == None: # no paragraph before stats table
    for line in page_block.text.split("\n"):
        print(line)
        line = line.strip()
        if line == "":
            continue
        print(line)
        if meta == None:
            meta = line
        if re.match(r'.*Armor\s+Class', line):
            AC = get_AC(line)
        elif re.match(r'.*Hit\s+Points', line):
            HP = get_HP(line)
        elif re.match(r'.*ft\.', line) or re.match(r'.*feet:
            movement = line
        elif re.match(r'.*Ability', line):
            break;
else:
    first_paragraph = main_block.find("div", id="page-content").find("p")
    fp_lines = first_paragraph.text.split("\n")
    meta = fp_lines[0]
    AC = get_AC(fp_lines[1])
    HP = get_HP(fp_lines[2])
    movement = fp_lines[3]

rows = stats_table.find_all("tr")

def get_stat(row):
    tds = row.find_all("td")
    return f'{tds[1].text} ({tds[2].text})'

stats = [get_stat(row) for row in rows[1:7]]
second_paragraph = stats_table.findNext("p")
tidbits = second_paragraph.text.replace("Skills ", "").replace("Senses ","").replace("Languages ","").replace("Challenge", "CR").replace("—","").replace("\n\n","\n").replace("\n","<br>").replace(" ft.","'")
tidbits = re.sub(r'Passive Perception [0-9]*<br>', "", tidbits)

third_paragraph = second_paragraph.findNext("p")
attributes = third_paragraph.text.replace(" ft.","'")



Tiny beast, unaligned
Tiny beast, unaligned
Armor Class 12
Armor Class 12
Hit Points 1 (1d4 - 1)
Hit Points 1 (1d4 - 1)
Speed 20 ft., climb 20 ft.
Speed 20 ft., climb 20 ft.


Ability
Ability


In [21]:
(AC,HP,movement,meta)

('12', None, None, 'Tiny beast, unaligned')